In [55]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [56]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

In [5]:
def NationalTeamCareer(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')
    
    national_team_career_section = soup.find('div', class_="box", attrs={"data-viewport": "Laenderspielkarriere"})
    national_team_data = []
    if national_team_career_section:
        for row in national_team_career_section.find_all('div', class_="national-career__row"):
            cells = row.find_all('div', class_="grid__cell")
            if cells:
                status_class = cells[0].get('class', [])
                status = ('Not Playing' if 'national-career__cell--red' in status_class 
                          else 'Playing')

                team_level = cells[1].get_text(strip=True)
                debut = cells[2].get_text(strip=True)
                matches = cells[3].get_text(strip=True)
                goals = cells[4].get_text(strip=True)

                national_team_data.append({
                    "National Status": status,
                    "National Team Level": team_level,
                    "National Debut": debut
                })

    
    latest_national_team_data = national_team_data[0] if national_team_data else {}
    national_player_profile_link = soup.find('a', text="Go to national player profile")
    national_player_profile_url = f"https://www.transfermarkt.com{national_player_profile_link['href']}" if national_player_profile_link else "Not available"

    return latest_national_team_data, national_player_profile_url

In [4]:
def scrape_national_team_stats(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extracting the name of the national team
    team_name_section = soup.find('div', class_='dataName')
    team_name = team_name_section.find('span', class_='hide-for-small').text.strip() if team_name_section else 'Unknown'

    # Find a table with data
    stats_table = soup.find('table', class_='items')
    player_stats = {}
    if stats_table:
        # Extract a line with general statistics
        footer = stats_table.find('tfoot')
        if footer:
            totals = footer.find('tr')
            cols = totals.find_all('td')
            if len(cols) >= 9:
                player_stats = {
                    'National Matches': cols[2].text.strip(),
                    'National Goals': cols[3].text.strip(),
                    'National Assists': cols[4].text.strip(),
                    'National Yellow Cards': cols[5].text.strip(),
                    'National Red Cards': cols[7].text.strip(),
                }

    return player_stats

нужна url профиля игрока. Она затем сама парсит National_profile и более детальную инфу берет 

In [6]:
# this cell is for one url

url = 'https://www.transfermarkt.com/neymar/profil/spieler/68290'
national_team_career_data, profile_url = NationalTeamCareer(url)
national_team_stats = scrape_national_team_stats(profile_url)


combined_data = {**national_team_career_data, **national_team_stats}
df = pd.DataFrame([combined_data])

df

,National Status,National Team Level,National Debut,National Matches,National Goals,National Assists,National Yellow Cards,National Red Cards
0,Playing,Brazil,"Aug 10, 2010",128,79,59,32,1


In [7]:
# and here you can add several links to the list, and you will get a beautiful sign
# but these are just indexes, you can also write names

urls = [
    'https://www.transfermarkt.com/neymar/profil/spieler/68290',
    'https://www.transfermarkt.com/rafa-marin/profil/spieler/662261',
    'https://www.transfermarkt.com/axel-witsel/profil/spieler/35047'
]

players_data = []

for url in urls:
    national_team_career_data, profile_url = NationalTeamCareer(url)
    national_team_stats = scrape_national_team_stats(profile_url)

    combined_data = {**national_team_career_data, **national_team_stats}
    players_data.append(combined_data)

df = pd.DataFrame(players_data)
df


,National Status,National Team Level,National Debut,National Matches,National Goals,National Assists,National Yellow Cards,National Red Cards
0,Playing,Brazil,"Aug 10, 2010",128,79,59,32,1
1,Playing,Spain U21,"Sep 8, 2023",6,-,-,1,-
2,Not Playing,Belgium,"Mar 26, 2008",130,12,8,14,2


* первая ссылка про Неймара, который является действующим игроком сборной
* вторая, про парнишу из U21 Испании
* третья про Витселя, который уже не играет за сборную

* the first link is about Neymar, who is a current national team player
* second, about a guy from U21 Spain
* the third is about Witsel, who no longer plays for the national team

## UPDATE status(0/1/2)

Теперь есть колонка статус, которая указывает:
* 0: Not Playing
* 1: Playing in a youth national team (U16, U17, U18, U19, U20, etc.)
* 2: Playing in the senior national team

There is now a status column that indicates:
* 0: Not Playing
* 1: Playing in a youth national team (U16, U17, U18, U19, U20, etc.)
* 2: Playing in the senior national team

In [62]:
def NationalTeamCareer(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')
    
    national_team_data = []
    national_player_profile_url = "Not available"
    
    national_team_career_section = soup.find('div', class_="box", attrs={"data-viewport": "Laenderspielkarriere"})
    if national_team_career_section:
        for row in national_team_career_section.find_all('div', class_="national-career__row"):
            cells = row.find_all('div', class_="grid__cell")
            if cells:
                status_class = cells[0].get('class', [])
                status_code = 0 if 'national-career__cell--red' in status_class else 2
                team_level = cells[1].get_text(strip=True)
                team_name = re.sub(r' U\d+$', '', team_level)
                if ' U' in team_level:
                    status_code = 1

                national_team_data.append({
                    "National Team": team_name,
                    "National Status": status_code,
                })

    if national_team_data:
        latest_national_team_data = national_team_data[0]
        national_player_profile_link = soup.find('a', text="Go to national player profile")
        if national_player_profile_link and national_player_profile_link.get('href'):
            national_player_profile_url = f"https://www.transfermarkt.com{national_player_profile_link['href']}"
    else:
        latest_national_team_data = {"National Team": "Unknown", "National Status": 0}

    return latest_national_team_data, national_player_profile_url

In [63]:
def scrape_national_team_stats(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    team_name_section = soup.find('div', class_='dataName')
    team_name = team_name_section.find('span', class_='hide-for-small').text.strip() if team_name_section else 'Unknown'

    stats_table = soup.find('table', class_='items')
    player_stats = {}
    if stats_table:
        footer = stats_table.find('tfoot')
        if footer:
            totals = footer.find('tr')
            cols = totals.find_all('td')
            if len(cols) >= 9:
                player_stats = {
                    'National Matches': cols[2].text.strip(),
                    'National Goals': cols[3].text.strip(),
                    'National Assists': cols[4].text.strip(),
                    'National Yellow Cards': cols[5].text.strip(),
                    'National Red Cards': cols[7].text.strip(),
                }

    return player_stats

In [67]:
# and here you can add several links to the list, and you will get a beautiful sign
# but these are just indexes, you can also write names

urls = [
    'https://www.transfermarkt.com/neymar/profil/spieler/68290', # Неймар играет за сборную Бразилии
    'https://www.transfermarkt.com/cristiano-ronaldo/profil/spieler/8198', # CR7 за Португалию
    'https://www.transfermarkt.com/rafa-marin/profil/spieler/662261', # Молодой Испанец за U21
    'https://www.transfermarkt.com/brahim-traore/profil/spieler/743517', # Б.Трауре молодой француз из U20
    'https://www.transfermarkt.com/axel-witsel/profil/spieler/35047', # Витсель завершил игры за сборную Белгии
    'https://www.transfermarkt.com/karim-benzema/profil/spieler/18922' # Бензема, которого выгнали из сборной Франции
    
]

players_data = []

for url in urls:
    national_team_career_data, profile_url = NationalTeamCareer(url)
    national_team_stats = scrape_national_team_stats(profile_url)

    combined_data = {**national_team_career_data, **national_team_stats}
    players_data.append(combined_data)

df = pd.DataFrame(players_data)
df




,National Team,National Status,National Matches,National Goals,National Assists,National Yellow Cards,National Red Cards
0,Brazil,2,128,79,59,32,1
1,Portugal,2,205,128,46,30,-
2,Spain,1,6,-,-,1,-
3,France,1,3,-,-,1,-
4,Belgium,0,130,12,8,14,2
5,France,0,97,37,20,1,-


In [68]:
# Replace nan with zeros

for player in players_data:
    for key, value in player.items():
        if value == '-':
            player[key] = 0 

df = pd.DataFrame(players_data)
df

,National Team,National Status,National Matches,National Goals,National Assists,National Yellow Cards,National Red Cards
0,Brazil,2,128,79,59,32,1
1,Portugal,2,205,128,46,30,0
2,Spain,1,6,0,0,1,0
3,France,1,3,0,0,1,0
4,Belgium,0,130,12,8,14,2
5,France,0,97,37,20,1,0
